# Extraction de mails 

Ce notebook à pour objectif d'extraire les mails reçus d'une boite mail pendant une période donnée.
Il prend en entrée les coordonnées de la boite mails et la période souhaitée et donnée en sortie un fichier excel contenant les mails

In [1]:
import warnings
warnings.filterwarnings("ignore")

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import exchangelib
from exchangelib import DELEGATE, Account, Credentials, Configuration

import datetime
from datetime import datetime
from datetime import date

In [2]:
## Les cordonnées de la boite mails

creds = Credentials(
    username="massermouh@groupe-idea.com", 
    password="0idcili1A*"
)

config = Configuration(server='outlook.office.com', credentials=creds)

account = Account(
    primary_smtp_address="n16aeroparc@groupe-idea.com",
    autodiscover=False, 
    config=config,
    access_type=DELEGATE
)

## Vous voulez extraire les mails reçus depuis quelle dates?

date_début = date(2021, 1, 1)
now = datetime.now().date()
delta = now - date_début
h = delta.days*24

In [4]:
def html_to_text(html):
    
    soup = BeautifulSoup(html, features="html.parser")

    for script in soup(["script", "style"]):
        script.extract()  
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    return text

In [ ]:
from datetime import timedelta
from exchangelib import UTC_NOW

mails = pd.DataFrame(columns=['Subject','Date_Sent','Body','From (address)','To (address)'])
                      
                      
since = UTC_NOW() - timedelta(hours=h)
for item in account.inbox.all()\
        .filter(datetime_received__gt=since)\
        .order_by('-datetime_received'):
    l=[]
    l.append(item.subject)
    l.append(item.datetime_received)
    b = html_to_text(item.unique_body)
    b = b.replace('\n',' ')
    l.append(b)
    l.append(item.sender.email_address)
    l.append(item.display_to)
    mails.loc[len(mails)] = l
    
# extracting mails in inbox subfolders

# sub_sub_folder = account.inbox / 'Litiges'
# for item in sub_sub_folder.all()\
#         .filter(datetime_received__gt=since)\
#         .order_by('-datetime_received'):

#         l=[]
#         l.append(item.subject)
#         l.append(item.datetime_received)
#         b = html_to_text(item.unique_body)
#         b = b.replace('\n',' ')
#         l.append(b)
#         l.append(item.sender.email_address)
#         l.append(item.display_to)

#         mails.loc[len(mails)] = l
        


    
mails['Date_Sent'] = pd.to_datetime(mails.Date_Sent).dt.tz_localize(None)

mails = mails.sort_values(by='Date_Sent')

In [ ]:
len(mails)

In [13]:
mails

,Subject,Date_Sent,Body,From (address),To (address)
0,Non remis : [à traiter] NS16 | D5313221000000 ...,2022-08-09 08:26:06,Your message to IMCEAEX-_o=idea_ou=Exchange+20...,MicrosoftExchange329e71ec88ae4615bbc36ab6ce411...,Granier Laurent
1,[à traiter] NS16 | D5313221000000 | DSU-ES-202...,2022-08-09 08:26:03,[Email émis par un expéditeur externe au group...,noreply@airbus.com,claude.passard@airbus.com; n16aeroparc; christ...
2,Problème avec BROF reçu (BROF n° d'avis 209179...,2022-08-09 07:42:38,Reçu hier un BROF avec 2 pièces au lieu d’une ...,scouton@groupe-idea.com,n16aeroparc
3,V5711056503000,2022-08-09 07:01:14,[Email émis par un expéditeur externe au group...,arthur.moingeon@airbus.com,n16aeroparc
4,[à traiter] NS16 | V5313211000000 | DSU-ES-202...,2022-08-08 09:37:48,[Email émis par un expéditeur externe au group...,noreply@airbus.com,claude.passard@airbus.com; n16aeroparc; christ...
...,...,...,...,...,...
67,Servitude lèvres A320 + visite Lisi,2021-10-08 12:28:11,"Bonjour, Concernant la servitude des lèvres A3...",ghislain.gaudin@airbus.com,n16aeroparc
68,TR: BOS,2021-09-09 09:18:34,"Cordialement, Anthony GOUPILLEAU Ingénieur Sup...",agoupilleau@groupe-idea.com,n16aeroparc
69,Re: /!\ FLASH Bonne Pratique réponse DAS,2021-06-16 11:59:13,"Une coquille s'est glissée dans mon document, ...",christophe.c.jarnet.external@airbus.com,n18aeroparc; n16aeroparc; Guillaume DENION; Ja...
70,Fwd: Litiges ASN potentiels,2021-06-14 10:07:12,Bonjour anthony pourrez vous regarder dans la ...,antoine.percevault@airbus.com,Goupilleau Anthony


In [21]:
# mails = mails[mails['Date_Sent'].dt.month <= 7] 

In [11]:
#mails.iloc[2]['Body']

In [22]:
mails.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 8 to 51
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Subject         44 non-null     object        
 1   Date_Sent       44 non-null     datetime64[ns]
 2   Body            44 non-null     object        
 3   From (address)  44 non-null     object        
 4   To (address)    44 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.1+ KB


In [23]:
output_file = "./Documents/Priorisation de mails/database_annotation.xlsx"
mails.to_excel(output_file)